# Sorting multiple particle types at once

This notebook is a follow-up to [Zonemaps](Zonemaps.ipynb).

Zonemaps let us skip over groups of events that don't have the particles we're looking for, but unless the cuts eliminate more than a factor of 1024 or so (i.e. 4 kB pages / 4-byte floats), those pages will still be loaded from disk into DRAM and/or from DRAM into CPU cache. That's because touching data anywhere on the page loads the whole page. Very deep cuts (more than a factor of ~1024) have a chance of missing whole pages, but shallow cuts, like one that drops 90% of the events, would only be skipping 90% of predicate-evaluations, not data-loading.

To make the zonemaps more effective, we can additionally sort the events. Sorting implies only one attribute, but $p_T$ is a very natural choice in HEP. If we sort events by particle $p_T$ and then use zonemaps to skip events by $p_T$ and other attributes (second and third highest $p_T$ per event are considered "other attributes"), then all of the hits will be concentrated at one end of the array. Fewer pages will be unnecessarily loaded, even for very shallow cuts: a cut that eliminates only half of the data will load roughly half of the pages, and that could be a factor-of-two win.

However, which particle's $p_T$? Different physics groups would be interested in different particles, and sorting by jet $p_T$ would improve jet access while worsening muon access.

In this notebook, I demonstrate a feature I added to PLUR: different particles can be sorted— event-wise— in different orders. This sorting is internal: the user sees events in the original order, but each set of particle arrays is laid out in an order that is sorted by that particle's $p_T$. The event pointer has to jump non-sequentially to access the sorted data.

It remains to be seen whether it's better to avoid reading unnecessary pages or to have sequential access to the particles. (If reading is from disk, I'd expect non-sequential access to be a smaller penalty.)

In [1]:
from math import *

import numpy

from plur.types import *
from plur.python.data import *

First, we load some data as before.

In [2]:
# pick a sample; naturally, this link to my home directory won't work for you
directory = "/home/pivarski/storage/data/TTJets_allevents/0/"

# "AK4CHS" are jets, "Muon" are muons
arrays = {n: numpy.load(directory + n + ".npy") for n in ("events-Lo", "events-Ld-R_Muon-Lo", "events-Ld-R_Muon-Ld-R_pt", "events-Ld-R_Muon-Ld-R_eta", "events-Ld-R_Muon-Ld-R_phi", "events-Ld-R_AK4CHS-Lo", "events-Ld-R_AK4CHS-Ld-R_pt", "events-Ld-R_AK4CHS-Ld-R_eta", "events-Ld-R_AK4CHS-Ld-R_phi")}

# look at the names of what we picked
sorted(arrays.keys(), reverse=True)

['events-Lo',
 'events-Ld-R_Muon-Lo',
 'events-Ld-R_Muon-Ld-R_pt',
 'events-Ld-R_Muon-Ld-R_phi',
 'events-Ld-R_Muon-Ld-R_eta',
 'events-Ld-R_AK4CHS-Lo',
 'events-Ld-R_AK4CHS-Ld-R_pt',
 'events-Ld-R_AK4CHS-Ld-R_phi',
 'events-Ld-R_AK4CHS-Ld-R_eta']

Print out a few of them. The "`at 0x0`", "`at 0x1`", "`at 0x2`", etc. in each string representation is the index that each proxy uses to find data in the arrays. Python objects typically show their pointer addresses, which are large, non-sequential numbers. These numbers are small and sequential because, by default, we start at the beginnings of the arrays and continue sequentially to the end.

In [3]:
events = fromarrays("events", arrays)
for event in events[:4]:
    print("event: " + repr(event))
    print("    muons: " + " ".join(repr(muon) for muon in event.Muon))
    print("    jets: " + " ".join(repr(jet) for jet in event.AK4CHS))
    print("")

event: <events at 0x0>
    muons: <Muon at 0x0>
    jets: <AK4CHS at 0x0> <AK4CHS at 0x1> <AK4CHS at 0x2> <AK4CHS at 0x3> <AK4CHS at 0x4> <AK4CHS at 0x5>

event: <events at 0x1>
    muons: 
    jets: <AK4CHS at 0x6> <AK4CHS at 0x7> <AK4CHS at 0x8> <AK4CHS at 0x9> <AK4CHS at 0xa> <AK4CHS at 0xb> <AK4CHS at 0xc>

event: <events at 0x2>
    muons: <Muon at 0x1> <Muon at 0x2> <Muon at 0x3>
    jets: <AK4CHS at 0xd> <AK4CHS at 0xe> <AK4CHS at 0xf> <AK4CHS at 0x10> <AK4CHS at 0x11> <AK4CHS at 0x12> <AK4CHS at 0x13> <AK4CHS at 0x14> <AK4CHS at 0x15> <AK4CHS at 0x16>

event: <events at 0x3>
    muons: <Muon at 0x4> <Muon at 0x5> <Muon at 0x6>
    jets: <AK4CHS at 0x17> <AK4CHS at 0x18> <AK4CHS at 0x19> <AK4CHS at 0x1a> <AK4CHS at 0x1b> <AK4CHS at 0x1c> <AK4CHS at 0x1d>



## Sorting arrays

Now get data to sort the arrays. We need one value per event, and the highest particle $p_T$ (for each type of particle) is the natural choice.

In [4]:
muonpt1 = numpy.zeros(len(events), dtype=numpy.float32)
jetpt1 = numpy.zeros(len(events), dtype=numpy.float32)

for eventi, event in enumerate(events):
    for muon in event.Muon:
        if muon.pt > muonpt1[eventi]:
            muonpt1[eventi] = muon.pt     # assume muons are unsorted (I don't know for sure)

    for jet in event.AK4CHS:
        if jet.pt > jetpt1[eventi]:
            jetpt1[eventi] = jet.pt       # assume jets are unsorted (I don't know for sure)

Get an argsort so that we can apply the same sort-order to all attributes in the particle tables. One argsort per particle type.

In [5]:
muonorder = muonpt1.argsort()[::-1]   # prefer max to min ordering
jetorder = jetpt1.argsort()[::-1]

print(muonorder[:10])
print(jetorder[:10])

[ 3582  9878 25486 37848 15068  9285 18487  8826 22319  9612]
[12665 19515 10279 21718 34930 33891 20522 39713 28183  5027]


To verify that this worked, look at the $p_T$ of all particles in the event with highest $p_T$ (separately for each particle). You should see at least one huge value— thousands (of GeV) is huge. The other particles in this even don't necessarily have huge $p_T$ values, though they might.

In [6]:
print([muon.pt for muon in events[muonorder[0]].Muon])
print([jet.pt for jet in events[jetorder[0]].AK4CHS])

[27280.783, 20.993748, 6.5631175, 3.9093113, 3.6213717, 3.1027567]
[1408.762, 1274.2667, 656.65344, 88.645607, 57.03783, 30.398226, 28.817825]


Now here's the tricky part. We have an argsort for attributes of the _event_ (highest $p_T$ per event), but we need an expanded argsort for all particles in each event.

The expanded argsort has the same number of elements as particles (or equivalently, the $p_T$ array). We want this sort order to move all particles in an event like the event-level argsort, but keep all particles together. We have to use for loops to do this (which can be accelerated by PLUR; don't worry about the speed of the code below).

In [7]:
muonorder_expanded = numpy.empty(len(arrays["events-Ld-R_Muon-Ld-R_pt"]), dtype=muonorder.dtype)
muonbegin = numpy.empty(len(events), dtype=numpy.int64)
muonend = numpy.empty(len(events), dtype=numpy.int64)

index = 0
for sortedi in muonorder:
    muonbegin[sortedi] = index
    for j in range(len(events[sortedi].Muon)):
        muonorder_expanded[index] = arrays["events-Ld-R_Muon-Lo"][sortedi] + j
        index += 1
    muonend[sortedi] = index

jetorder_expanded = numpy.empty(len(arrays["events-Ld-R_AK4CHS-Ld-R_pt"]), dtype=jetorder.dtype)
jetbegin = numpy.empty(len(events), dtype=numpy.int64)
jetend = numpy.empty(len(events), dtype=numpy.int64)

index = 0
for sortedi in jetorder:
    jetbegin[sortedi] = index
    for j in range(len(events[sortedi].AK4CHS)):
        jetorder_expanded[index] = arrays["events-Ld-R_AK4CHS-Lo"][sortedi] + j
        index += 1
    jetend[sortedi] = index

Now we make a new dataset of sorted data. Each of the muon's attributes is rearranged by the `muonorder_expanded`, and each of the jet's attributes is rearranged by the `jetorder_expanded`.

Also, instead of "list offset" arrays (ending in `-Lo`), we now have "list begin" and "list end" arrays (ending in `-Lb` and `-Le`). The new feature in PLUR is that we can have different begins and ends, which allows a sequential traversal over logical events to be non-sequential in the underlying arrays.

In [8]:
sortedarrays = {
    "sortedevents-Lo":                   arrays["events-Lo"],
    "sortedevents-Ld-R_Muon-Lb":         muonbegin,
    "sortedevents-Ld-R_Muon-Le":         muonend,
    "sortedevents-Ld-R_Muon-Ld-R_pt":    arrays["events-Ld-R_Muon-Ld-R_pt"][muonorder_expanded],
    "sortedevents-Ld-R_Muon-Ld-R_phi":   arrays["events-Ld-R_Muon-Ld-R_phi"][muonorder_expanded],
    "sortedevents-Ld-R_Muon-Ld-R_eta":   arrays["events-Ld-R_Muon-Ld-R_eta"][muonorder_expanded],
    "sortedevents-Ld-R_AK4CHS-Lb":       jetbegin,
    "sortedevents-Ld-R_AK4CHS-Le":       jetend,
    "sortedevents-Ld-R_AK4CHS-Ld-R_pt":  arrays["events-Ld-R_AK4CHS-Ld-R_pt"][jetorder_expanded],
    "sortedevents-Ld-R_AK4CHS-Ld-R_phi": arrays["events-Ld-R_AK4CHS-Ld-R_phi"][jetorder_expanded],
    "sortedevents-Ld-R_AK4CHS-Ld-R_eta": arrays["events-Ld-R_AK4CHS-Ld-R_eta"][jetorder_expanded]}

sortedevents = fromarrays("sortedevents", sortedarrays)

Just to see that this worked, print out the first (logical) event in `events` and `sortedevents`.

Both collections have the same contents, as seen by the user.

In [9]:
which = 0
print("original:\n" + "\n".join(repr((muon.pt, muon.phi, muon.eta)) for muon in events[which].Muon) + "\n")
print("sorted:  \n" + "\n".join(repr((muon.pt, muon.phi, muon.eta)) for muon in sortedevents[which].Muon))

original:
(28.070749, -2.5696332, -1.3311582)

sorted:  
(28.070749, -2.5696332, -1.3311582)


Same for the event with highest muon $p_T$.

In [10]:
which = muonorder[0]
print("original:\n" + "\n".join(repr((muon.pt, muon.phi, muon.eta)) for muon in events[which].Muon) + "\n")
print("sorted:  \n" + "\n".join(repr((muon.pt, muon.phi, muon.eta)) for muon in sortedevents[which].Muon))

original:
(27280.783, 1.9008816, -1.4215664)
(20.993748, 2.0672431, -1.2065552)
(6.5631175, -0.78409564, 0.39030939)
(3.9093113, 0.42742085, 0.53176123)
(3.6213717, 2.8320415, -1.7658416)
(3.1027567, -0.29853693, -1.5238833)

sorted:  
(27280.783, 1.9008816, -1.4215664)
(20.993748, 2.0672431, -1.2065552)
(6.5631175, -0.78409564, 0.39030939)
(3.9093113, 0.42742085, 0.53176123)
(3.6213717, 2.8320415, -1.7658416)
(3.1027567, -0.29853693, -1.5238833)


The first event, looing at jets now.

In [11]:
which = 0
print("original:\n" + "\n".join(repr((jet.pt, jet.phi, jet.eta)) for jet in events[which].AK4CHS) + "\n")
print("sorted:  \n" + "\n".join(repr((jet.pt, jet.phi, jet.eta)) for jet in sortedevents[which].AK4CHS))

original:
(194.19714, 0.40622151, -2.6520884)
(136.11476, -2.528533, -1.3676349)
(85.622955, 2.8486614, -0.60089219)
(60.950535, -0.94596076, -3.3134294)
(18.965042, -0.07733579, -0.8700369)
(18.361681, 2.1469259, 2.6447577)

sorted:  
(194.19714, 0.40622151, -2.6520884)
(136.11476, -2.528533, -1.3676349)
(85.622955, 2.8486614, -0.60089219)
(60.950535, -0.94596076, -3.3134294)
(18.965042, -0.07733579, -0.8700369)
(18.361681, 2.1469259, 2.6447577)


Same for the event with highest jet $p_T$.

In [12]:
which = jetorder[0]
print("original:\n" + "\n".join(repr((jet.pt, jet.phi, jet.eta)) for jet in events[which].AK4CHS) + "\n")
print("sorted:  \n" + "\n".join(repr((jet.pt, jet.phi, jet.eta)) for jet in sortedevents[which].AK4CHS))

original:
(1408.762, 1.6171466, -0.80842471)
(1274.2667, -2.1024816, 0.31906798)
(656.65344, -0.20202568, 0.47772574)
(88.645607, -2.1789863, -0.38160208)
(57.03783, -0.84209001, 0.20092875)
(30.398226, -2.4329026, 1.2096163)
(28.817825, -0.99910033, 1.0436321)

sorted:  
(1408.762, 1.6171466, -0.80842471)
(1274.2667, -2.1024816, 0.31906798)
(656.65344, -0.20202568, 0.47772574)
(88.645607, -2.1789863, -0.38160208)
(57.03783, -0.84209001, 0.20092875)
(30.398226, -2.4329026, 1.2096163)
(28.817825, -0.99910033, 1.0436321)


But now if we look at the physical arrangement of muon data, we see that the event with the highest $p_T$ muon is first.

In [13]:
sortedarrays["sortedevents-Ld-R_Muon-Ld-R_pt"][:10].tolist()

[27280.783203125,
 20.99374771118164,
 6.563117504119873,
 3.909311294555664,
 3.6213717460632324,
 3.1027567386627197,
 20537.115234375,
 3.7412970066070557,
 11355.5908203125,
 48.06831741333008]

Same for jets, which implies a different sort order.

In [14]:
sortedarrays["sortedevents-Ld-R_AK4CHS-Ld-R_pt"][:10].tolist()

[1408.761962890625,
 1274.2667236328125,
 656.6534423828125,
 88.6456069946289,
 57.0378303527832,
 30.398225784301758,
 28.817825317382812,
 1184.8544921875,
 927.7994995117188,
 131.87158203125]

Finally, if we look at the first four events in the `sortedevents` dataset, we see that the first indexes are no longer small or sequential across events.

They are, however, sequential _within_ each event.

In [15]:
for event in sortedevents[:4]:
    print("event: " + repr(event))
    print("    muons: " + " ".join(repr(muon) for muon in event.Muon))
    print("    jets: " + " ".join(repr(jet) for jet in event.AK4CHS))
    print("")

event: <sortedevents at 0x0>
    muons: <Muon at 0x44bf>
    jets: <AK4CHS at 0xadd2> <AK4CHS at 0xadd3> <AK4CHS at 0xadd4> <AK4CHS at 0xadd5> <AK4CHS at 0xadd6> <AK4CHS at 0xadd7>

event: <sortedevents at 0x1>
    muons: 
    jets: <AK4CHS at 0x32e4f> <AK4CHS at 0x32e50> <AK4CHS at 0x32e51> <AK4CHS at 0x32e52> <AK4CHS at 0x32e53> <AK4CHS at 0x32e54> <AK4CHS at 0x32e55>

event: <sortedevents at 0x2>
    muons: <Muon at 0x9f76> <Muon at 0x9f77> <Muon at 0x9f78>
    jets: <AK4CHS at 0x310d1> <AK4CHS at 0x310d2> <AK4CHS at 0x310d3> <AK4CHS at 0x310d4> <AK4CHS at 0x310d5> <AK4CHS at 0x310d6> <AK4CHS at 0x310d7> <AK4CHS at 0x310d8> <AK4CHS at 0x310d9> <AK4CHS at 0x310da>

event: <sortedevents at 0x3>
    muons: <Muon at 0x25d9> <Muon at 0x25da> <Muon at 0x25db>
    jets: <AK4CHS at 0x10758> <AK4CHS at 0x10759> <AK4CHS at 0x1075a> <AK4CHS at 0x1075b> <AK4CHS at 0x1075c> <AK4CHS at 0x1075d> <AK4CHS at 0x1075e>

